In [2]:
import pandas as pd
import numpy as np
import polars as pl

# Imports

In [34]:
file_path_forecasts = '../data/dataset2014-2022-v3.zip'
file_path_company = '../data/dataset-company-industry-data-v1.csv'

df_forecasts = pd.read_csv(file_path_forecasts)
df_companies = pd.read_csv(file_path_company)

In [14]:
df_forecasts

,TICKER,CUSIP,OFTIC,CNAME,ESTIMATOR,ANALYS,FPI,MEASURE,VALUE,FPEDATS,REVDATS,ANNDATS,ACTUAL,ANNDATS_ACT
0,0000,87482X10,TLMR,TALMER BANCORP,2500,72481,1,EPS,1.200,2014-12-31,2014-12-03,2014-12-03,1.21,2015-01-30
1,0000,87482X10,TLMR,TALMER BANCORP,2500,72481,1,EPS,0.850,2015-12-31,2015-02-18,2015-02-18,1.02,2016-01-26
2,0000,87482X10,TLMR,TALMER BANCORP,149,119962,1,EPS,0.860,2015-12-31,2015-02-18,2015-02-18,1.02,2016-01-26
3,0000,87482X10,TLMR,TALMER BANCORP,1267,71182,1,EPS,0.890,2015-12-31,2015-04-30,2015-02-18,1.02,2016-01-26
4,0000,87482X10,TLMR,TALMER BANCORP,149,119962,1,EPS,0.890,2015-12-31,2015-04-30,2015-02-23,1.02,2016-01-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570994,ZYNX,98986M10,ZYXI,ZYNEX INC,2709,122730,1,EPS,0.460,2022-12-31,2022-07-29,2022-07-29,0.44,2023-03-13
1570995,ZYNX,98986M10,ZYXI,ZYNEX INC,926,118308,1,EPS,0.420,2022-12-31,2023-01-18,2022-07-29,0.44,2023-03-13
1570996,ZYNX,98986M10,ZYXI,ZYNEX,220,193620,1,EPS,0.423,2022-12-31,2023-01-06,2022-10-27,0.44,2023-03-13
1570997,ZYNX,98986M10,ZYXI,ZYNEX,2709,122730,1,EPS,0.440,2022-12-31,2022-10-28,2022-10-28,0.44,2023-03-13


In [15]:
df_companies

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,curcd,fyr,costat,sic,spcindcd
0,1004,2014-05-31,2013.0,INDL,C,D,STD,AIR,000361105,USD,5.0,A,5080.0,110.0
1,1004,2015-05-31,2014.0,INDL,C,D,STD,AIR,000361105,USD,5.0,A,5080.0,110.0
2,1004,2016-05-31,2015.0,INDL,C,D,STD,AIR,000361105,USD,5.0,A,5080.0,110.0
3,1004,2017-05-31,2016.0,INDL,C,D,STD,AIR,000361105,USD,5.0,A,5080.0,110.0
4,1004,2018-05-31,2017.0,INDL,C,D,STD,AIR,000361105,USD,5.0,A,5080.0,110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115407,353742,2021-11-30,2021.0,INDL,C,D,STD,LIFFF,53000A106,CAD,11.0,A,1400.0,NaN
115408,353742,2022-11-30,2022.0,INDL,C,D,STD,LIFFF,53000A106,CAD,11.0,A,1400.0,NaN
115409,353945,2021-12-31,2021.0,INDL,C,D,STD,ACLLY,00449R109,USD,12.0,A,3621.0,NaN
115410,353945,2022-12-31,2022.0,INDL,C,D,STD,ACLLY,00449R109,USD,12.0,A,3621.0,NaN


# Merge SIC Codes 

In [18]:
df_forecasts.head()

,ticker,cusip,oftic,cname,estimator,analys,fpi,measure,value,fpedats,revdats,anndats,actual,anndats_act,difference_date
1,0000,87482X10,TLMR,TALMER BANCORP,2500,72481,1,EPS,0.85,2015-12-31,2015-02-18,2015-02-18,1.02,2016-01-26,316 days
2,0000,87482X10,TLMR,TALMER BANCORP,149,119962,1,EPS,0.86,2015-12-31,2015-02-18,2015-02-18,1.02,2016-01-26,316 days
3,0000,87482X10,TLMR,TALMER BANCORP,1267,71182,1,EPS,0.89,2015-12-31,2015-04-30,2015-02-18,1.02,2016-01-26,316 days
4,0000,87482X10,TLMR,TALMER BANCORP,149,119962,1,EPS,0.89,2015-12-31,2015-04-30,2015-02-23,1.02,2016-01-26,311 days
5,0000,87482X10,TLMR,TALMER BANCORP,873,79092,1,EPS,0.91,2015-12-31,2015-07-06,2015-04-07,1.02,2016-01-26,268 days


In [36]:
sic_codes = df_companies[['tic', 'sic']].drop_duplicates().reset_index(drop=True)

## Preprocessing

- Adjust datetime datatype columns
- Apply restrictions based on our assumptions
- Re-scaling depending on accuracy

In [42]:
# convert columns with dates to datetime
df_forecasts = df_forecasts.rename(columns={col: col.lower() for col in df_forecasts.columns})
def convert_to_datetime(df):
    columns_to_convert = ['fpedats','revdats', "anndats", 'anndats_act']
    for column in columns_to_convert:
        df[column] = pd.to_datetime(df[column])
    return df
df_forecasts = convert_to_datetime(df_forecasts)
# filter forecasts  based on time
df_forecasts['difference_date'] = df_forecasts['fpedats'] - df_forecasts['anndats']
df_forecasts = df_forecasts[(df_forecasts['difference_date']> pd.Timedelta(days=30))&(df_forecasts['difference_date']<pd.Timedelta(days=365))]
df_forecasts.head()

,ticker,cusip,oftic,cname,estimator,analys,fpi,measure,value,fpedats,revdats,anndats,actual,anndats_act,difference_date
1,0000,87482X10,TLMR,TALMER BANCORP,2500,72481,1,EPS,0.85,2015-12-31,2015-02-18,2015-02-18,1.02,2016-01-26,316 days
2,0000,87482X10,TLMR,TALMER BANCORP,149,119962,1,EPS,0.86,2015-12-31,2015-02-18,2015-02-18,1.02,2016-01-26,316 days
3,0000,87482X10,TLMR,TALMER BANCORP,1267,71182,1,EPS,0.89,2015-12-31,2015-04-30,2015-02-18,1.02,2016-01-26,316 days
4,0000,87482X10,TLMR,TALMER BANCORP,149,119962,1,EPS,0.89,2015-12-31,2015-04-30,2015-02-23,1.02,2016-01-26,311 days
5,0000,87482X10,TLMR,TALMER BANCORP,873,79092,1,EPS,0.91,2015-12-31,2015-07-06,2015-04-07,1.02,2016-01-26,268 days


### Data set restrictions

Restrictions should be implemented with respective functions

- Sample horizon 10 - 20 years
- Frequency: annual
- Include only forecasts issued no earlier than 1 year ahead and no later than 30 days before fiscal year end
- (Exclude forecasts for companies which only one analyst provides a forecast)
- Exclude all analysts appearing in the initial 2-3 years of the dataset (migitating problem of experience)
- (Retain the last forecast an analyst issues in a particular year)
- Drop all unneccessary columns from df
- ...

## Merge df with mapping list

In [43]:
df_merged = pd.merge(left=df_forecasts, right=sic_codes, how='left', left_on='oftic', right_on='tic')
df_merged = df_merged.drop(columns=['tic'])
df_merged.head()

,ticker,cusip,oftic,cname,estimator,analys,fpi,measure,value,fpedats,revdats,anndats,actual,anndats_act,difference_date,sic
0,0000,87482X10,TLMR,TALMER BANCORP,2500,72481,1,EPS,0.85,2015-12-31,2015-02-18,2015-02-18,1.02,2016-01-26,316 days,6020.0
1,0000,87482X10,TLMR,TALMER BANCORP,149,119962,1,EPS,0.86,2015-12-31,2015-02-18,2015-02-18,1.02,2016-01-26,316 days,6020.0
2,0000,87482X10,TLMR,TALMER BANCORP,1267,71182,1,EPS,0.89,2015-12-31,2015-04-30,2015-02-18,1.02,2016-01-26,316 days,6020.0
3,0000,87482X10,TLMR,TALMER BANCORP,149,119962,1,EPS,0.89,2015-12-31,2015-04-30,2015-02-23,1.02,2016-01-26,311 days,6020.0
4,0000,87482X10,TLMR,TALMER BANCORP,873,79092,1,EPS,0.91,2015-12-31,2015-07-06,2015-04-07,1.02,2016-01-26,268 days,6020.0


# DV and Feature Generation

Feature list (Proxies for analyst characteristics):

- Forecasting ability/experience: number of years for which analyst i supplied at least one forecast during the first 11 months of the year through year t
- Broker size: a dummy variable set to 1 if analyst i is employed by a firm in the top size decile during year t, and set to 0 otherwise. Size deciles are calculated based on the number of analysts employed in year t.
- Complexity: number of firms for which analyst i supplied at least one forecast during the first 11 months of year t OR number of two-digit SICs for which analyst i supplied at least one forecast during the first 11 months of year t minus the average number of two-digit SICs followed by an analyst following firm j at time t.
- Forecast age: age (in days) of analyst i's forecast for firm j's earnings at time t
- 
